This file Contains  the data request to the API.



In [ ]:
!pip install sodapy

In [ ]:
import pandas as pd
from sodapy import Socrata

import pandas as pd
from sodapy import Socrata

client = Socrata("www.datos.gov.co",
                  "tcU2fieqxe5EiHJE2OcqmSrPq")

In [ ]:
import pandas as pd

# Define dataset and base query (without LIMIT or OFFSET)
socrata_dataset_identifier = "jbjy-vk9h"

base_query = """
select
presupuesto_general_de_la_nacion_pgn,
recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_,
fecha_inicio_liquidacion,
fecha_fin_liquidacion,
objeto_del_contrato,
duraci_n_del_contrato,
urlproceso,
nombre_representante_legal,
origen_de_los_recursos,
liquidaci_n,
valor_del_contrato,
valor_facturado,
es_pyme,
proveedor_adjudicado,
fecha_de_fin_de_ejecucion,
fecha_de_inicio_de_ejecucion,
fecha_de_fin_del_contrato,
fecha_de_inicio_del_contrato,
fecha_de_firma,
modalidad_de_contratacion,
tipo_de_contrato,
codigo_de_categoria_principal,
descripcion_del_proceso,
estado_contrato,
referencia_del_contrato,
id_contrato,
proceso_de_compra,
entidad_centralizada,
rama,
sector,
orden,
localizaci_n,
ciudad,
departamento,
nit_entidad,
nombre_entidad
where
    fecha_de_firma >= '2024-01-01' AND fecha_de_firma < '2024-01-31'
    and estado_contrato not in ('En aprobación','enviado Proveedor','Borrador','Cancelado')
order by fecha_de_firma desc
"""

contratos_2024 = client.get(socrata_dataset_identifier, content_type="json", query=base_query)

df_contratos = pd.DataFrame(pd.DataFrame.from_dict(contratos_2024))
df_contratos

In [ ]:
#Changing the datatypes

import pandas as pd

# Assuming df_contratos is your DataFrame

# 1. List all text/string columns
text_columns = [
    'objeto_del_contrato',
    'duraci_n_del_contrato',
    'urlproceso',
    'nombre_representante_legal',
    'origen_de_los_recursos',
    'liquidaci_n',
    'es_pyme',
    'proveedor_adjudicado',
    'modalidad_de_contratacion',
    'tipo_de_contrato',
    'codigo_de_categoria_principal',
    'descripcion_del_proceso',
    'estado_contrato',
    'referencia_del_contrato',
    'id_contrato',
    'proceso_de_compra',
    'entidad_centralizada',
    'rama',
    'sector',
    'orden',
    'localizaci_n',
    'ciudad',
    'departamento',
    'nit_entidad',
    'nombre_entidad'
]

# 2. List all numeric (float) columns
numeric_columns = [
    'presupuesto_general_de_la_nacion_pgn',
    'recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
    'valor_del_contrato',
    'valor_facturado'
]

# 3. List all date columns
date_columns = [
    'fecha_inicio_liquidacion',
    'fecha_fin_liquidacion',
    'fecha_de_fin_de_ejecucion',
    'fecha_de_inicio_de_ejecucion',
    'fecha_de_fin_del_contrato',
    'fecha_de_inicio_del_contrato',
    'fecha_de_firma'
]

# Convert text columns to string
for col in text_columns:
    if col in df_contratos.columns:
        df_contratos[col] = df_contratos[col].astype(str)

# Convert numeric columns to float (handling errors)
for col in numeric_columns:
    if col in df_contratos.columns:
        df_contratos[col] = pd.to_numeric(df_contratos[col], errors='coerce')

# Convert date columns to datetime (handling errors)
for col in date_columns:
    if col in df_contratos.columns:
        df_contratos[col] = pd.to_datetime(df_contratos[col], errors='coerce')

# Show the final data types
print("Final Data Types:")
print(df_contratos.dtypes)

In [ ]:
# Install spaCy and Spanish model
!pip install -U spacy
!python -m spacy download es_core_news_sm


In [ ]:
## Cleaning, removing stop words
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy

# Load Spanish stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# 👉 Add domain-specific stopwords (based on your exploration)
contract_stopwords = {
    'prestacion', 'servicio', 'proceso', 'contratista', 'acuerdo', 'ejecutar',
    'condicion', 'empresa', 'nuevo', 'institucional', 'entidad', 'area',
    'eficaz', 'eficiente', 'municipio', 'nivel', 'centro', 'ejecución',
    'prestación', 'objeto', 'dentro', 'requerido', 'diferente', 'poner',
    'realizar',
    'apoyo', 'departamental', 'contrato', 'laboral', 'actividad', 'cuenta',
    'profesional', 'contratar', 'especializado', 'prestar', 'área',
    'subproceso', 'comprometer', 'garantizar', 'requerir', 'adelantar',
    'local', 'primero', 'conformidad', 'responsabilidad', 'efectivo',
    'disposición', 'forma', 'propuesta', 'bajo', 'oportuno', 'tiempo', 'autonomía'
}
stop_words.update(contract_stopwords)

# Load Spanish language model from spaCy
nlp = spacy.load("es_core_news_sm")

# Define full cleaning function with lemmatization and entity removal
def clean_lemma_no_entities(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if token.lemma_ not in stop_words
        and token.ent_type_ not in ("PER", "LOC", "ORG")  # Named entities
        and not token.is_punct and not token.is_space
    ]
    return ' '.join(tokens)

# Apply to your contract descriptions
df =df_contratos.copy()
df['objeto_clean_lemma'] = df['objeto_del_contrato'].apply(clean_lemma_no_entities)

In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")

def clean_and_lemmatize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct and not token.is_space]
    return ' '.join(lemmatized)

df['objeto_clean_lemma'] = df['objeto_del_contrato'].apply(clean_and_lemmatize)

In [ ]:
!pip install bertopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Use a multilingual model
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Create BERTopic model with custom embeddings
topic_model = BERTopic(embedding_model=embedding_model, language="spanish", verbose=True)

docs = df['objeto_clean_lemma'].tolist()

topics, probs= topic_model.fit_transform(docs)##Storing the probabilities
topic_model.get_topic_info()

In [ ]:
 !pip install umap-learn

In [ ]:
import umap
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
docs = df['objeto_clean_lemma'].tolist()
embeddings = embedding_model.encode(docs, show_progress_bar=True)

n_neighbors_values = [5, 15, 30, 50]  # Try different levels of locality
results = []

for n in n_neighbors_values:
    umap_model = umap.UMAP(n_neighbors=n, min_dist=0.5, random_state=42)
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        language="spanish",
        verbose=False
    )

    topics, _ = topic_model.fit_transform(docs, embeddings)
    n_topics = len(set(topics)) - (1 if -1 in topics else 0)
    results.append((n, n_topics))
    print(f"n_neighbors={n} → {n_topics} topics")

import umap
import matplotlib.pyplot as plt

n_neighbors_list = [2, 5, 10, 15, 20, 30, 50, 100, 150]
min_dist_fixed = 0.5

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle("UMAP - Varying n_neighbors (min_dist=0.5)", fontsize=16)

for i, n in enumerate(n_neighbors_list):
    reducer = umap.UMAP(n_neighbors=n, min_dist=min_dist_fixed, random_state=42)
    umap_result = reducer.fit_transform(embeddings)

    row, col = divmod(i, 3)
    ax = axes[row][col]
    ax.scatter(umap_result[:, 0], umap_result[:, 1], s=10)
    ax.set_title(f"n={n}")
    ax.axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
min_dist_values = [0.0, 0.1, 0.3, 0.5, 0.8]
results_min_dist = []

for d in min_dist_values:
    umap_model = umap.UMAP(n_neighbors=2, min_dist=d, random_state=42)
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        language="spanish",
        verbose=False
    )

    topics, _ = topic_model.fit_transform(docs, embeddings)
    n_topics = len(set(topics)) - (1 if -1 in topics else 0)
    results_min_dist.append((d, n_topics))
    print(f"min_dist={d} → {n_topics} topics")

    min_dist_list = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99]
n_neighbors_fixed = 15

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("UMAP - Varying min_dist (n_neighbors=10)", fontsize=16)

for i, d in enumerate(min_dist_list):
    reducer = umap.UMAP(n_neighbors=n_neighbors_fixed, min_dist=d, random_state=42)
    umap_result = reducer.fit_transform(embeddings)

    row, col = divmod(i, 4)
    ax = axes[row][col]
    ax.scatter(umap_result[:, 0], umap_result[:, 1], s=10)
    ax.set_title(f"min_dist={d}")
    ax.axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot n_neighbors results
x, y = zip(*results)
plt.figure()
plt.plot(x, y, marker='o')
plt.title("Effect of n_neighbors on Number of Topics")
plt.xlabel("n_neighbors")
plt.ylabel("Number of Topics")
plt.grid(True)
plt.show()

# Plot min_dist results
x2, y2 = zip(*results_min_dist)
plt.figure()
plt.plot(x2, y2, marker='o', color='orange')
plt.title("Effect of min_dist on Number of Topics")
plt.xlabel("min_dist")
plt.ylabel("Number of Topics")
plt.grid(True)
plt.show()


import umap

final_umap = umap.UMAP(n_neighbors=10, min_dist=0.5, random_state=42)

In [ ]:
import umap
from hdbscan import HDBSCAN
from bertopic import BERTopic

hdbscan_model = HDBSCAN(
    min_cluster_size=10,         # allow smaller topics
    min_samples=3,               # be tolerant with outliers
    prediction_data=True,
    cluster_selection_method='leaf'  # allow finer subclusters
)

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=final_umap,       # your best UMAP config
    hdbscan_model=hdbscan_model,
    language="spanish",
    verbose=True
)

topics, probs = topic_model.fit_transform(docs, embeddings)


In [ ]:
import numpy as np

num_outliers = np.sum(np.array(topics) == -1)
total = len(topics)
print(f"Outlier rate: {num_outliers}/{total} → {num_outliers / total:.2%}")


In [ ]:
df['topic_id'] = topics

In [ ]:
##Adding most representative keywords

topic_info = topic_model.get_topic_info()
topic_labels = {
    row['Topic']: row['Name'] for _, row in topic_info.iterrows()
}
df['topic_label'] = df['topic_id'].map(topic_labels)


In [ ]:
df['topic_label'].value_counts().plot(kind='bar', figsize=(10, 5), title="Contract Categories (BERTopic)")

In [ ]:
df.groupby(['ciudad', 'topic_label']).size().unstack(fill_value=0).head()

In [ ]:
df.to_excel("contratos_categorizados_bertopic_Unlimited.xlsx", index=False)